# Important!

* ## Running all cells will overload the open route service query limit per minute. 
This gives a bunch of warnings but will still be fine, or you can wait for a minute after running up to the first map and get no warnings
* ## Put your own key into the orsKey

In [1]:
import openrouteservice as ors
import numpy as np
import seaborn as sns
import folium
import pandas as pd
from Woolworths_LP_removed import *

# Please use your own key
orsKey = "5b3ce3597851110001cf6248048e866c6e2c4799ac7e290450acd146"


In [2]:

# Use the Woolworths_LP functions to get the optimal routes

Weekday_Routes, Weekend_Routes, storeLocations = load_data()


status, minimisedCost, routesWeek = solve_lp(Weekday_Routes, storeLocations)
status, minimisedCost, routesSat = solve_lp(Weekend_Routes, storeLocations, True)

In [3]:
# Add circle markers for each location

locations = pd.read_csv("WoolworthsLocations_removed.csv")
coords = locations[["Long", "Lat"]]
# Mapping   packages work with Long, Lat arrays
coords = coords.to_numpy().tolist()
# Make the arrays into a list of lists.
# Folium,however, requires Lat, Long arrays - so a reversal is needed.
# coords[0] is the warehouse

m = folium.Map(location=list(reversed(coords[2])), zoom_start=10)

for i in range(0, len(coords)):
    if locations.Type[i] == "Countdown":
        iconCol = "green"
    elif locations.Type[i] == "FreshChoice":
        iconCol = "blue"
    elif locations.Type[i] == "SuperValue":
        iconCol = "red"
    elif locations.Type[i] == "Countdown Metro":
        iconCol = "orange"
    elif locations.Type[i] == "Distribution Centre":
        iconCol = "black"
    folium.CircleMarker(
        list(reversed(coords[i])),
        popup=locations.Store[i],
        fill = True,
        color=iconCol,
        radius=2
    ).add_to(m)
m

In [4]:
client = ors.Client(key=orsKey)

In [5]:
# This takes the input routes and changes them into the correct format for open route service queries
# Since this runs twice probably better to make it a function

queryRoutes = [[coords[53]] for _ in range(len(routesWeek))]

for r in range(len(routesWeek)):
    for loc in range(2,6):
        for i in range(len(coords)):
            if routesWeek.iloc[r,loc]==locations.iloc[i, 2]:
                queryRoutes[r].append(coords[i])
                break
    queryRoutes[r].append(coords[53])





In [6]:
m = folium.Map(location=list(reversed(coords[2])), tiles='CartoDB positron', zoom_start=10)

# col is the list of colours for the lines. 
# Currently using a list of maximally dissimilar colours from internet so that routes are distinguishable. 


col = ["#000000", "#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059",
"#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
"#5A0007", "#809693", "#ABCDEF", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
"#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
"#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",]


# It would likely be better to save the routes rather than generating them and then forgetting them the next iteration of the loop
# Saving the routes to a file would mean we don't have issues with query frequency like we currently have

# Additionally if multiple routes follow the same road only one is visible, potentially can't be fixed.

for r in range(len(routesWeek)):
    # Query Open Route Service
    route = client.directions(coordinates=queryRoutes[r], 
                                profile = "driving-hgv", 
                                format="geojson", 
                                validate=False)

    # Add the route to the map
    folium.PolyLine(locations = [list(reversed(coord)) 
                                 for coord in 
                                 route["features"][0]["geometry"]["coordinates"]],
                                 color=col[r]).add_to(m)

for i in range(0, len(coords)):
    folium.CircleMarker(
        list(reversed(coords[i])),
        popup=locations.Store[i],
        fill = True,
        color="black",
        radius=2
    ).add_to(m)
m

In [7]:
# This takes the input routes and changes them into the correct format for open route service queries

queryRoutes = [[coords[53]] for _ in range(len(routesSat))]

for r in range(len(routesSat)):
    for loc in range(2,6):
        for i in range(len(coords)):
            if routesSat.iloc[r,loc]==locations.iloc[i, 2]:
                queryRoutes[r].append(coords[i])
                break
    queryRoutes[r].append(coords[53])

In [8]:
m = folium.Map(location=list(reversed(coords[2])), tiles='CartoDB positron', zoom_start=10)

# col is the list of colours for the lines. 
# Currently using a list of maximally dissimilar colours from internet so that routes are distinguishable. 
# These colours were not selected to be plotted on a map, so some of them may be difficult to see against the coloured background.


col = ["#000000", "#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059",
"#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
"#5A0007", "#809693", "#FEFFE6", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
"#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
"#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",]


# It would likely be better to save the routes rather than generating them and then forgetting them the next iteration of the loop
# Saving the routes to a file would mean we don't have issues with query frequency like we currently have

for r in range(len(routesSat)):
    route = client.directions(coordinates=queryRoutes[r], 
                                profile = "driving-hgv", 
                                format="geojson", 
                                validate=False)
    folium.PolyLine(locations = [list(reversed(coord)) 
                                 for coord in 
                                 route["features"][0]["geometry"]["coordinates"]],
                                 color=col[r]).add_to(m)

for i in range(0, len(coords)):
    folium.CircleMarker(
        list(reversed(coords[i])),
        popup=locations.Store[i],
        fill = True,
        color="black",
        radius=2
    ).add_to(m)

m

C:\Users\21sun\Documents\Engsci 263\Section 2\Project\ENGSCI-263-OR-Project\.venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\21sun\Documents\Engsci 263\Section 2\Project\ENGSCI-263-OR-Project\.venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\21sun\Documents\Engsci 263\Section 2\Project\ENGSCI-263-OR-Project\.venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\21sun\Documents\Engsci 263\Section 2\Project\ENGSCI-263-OR-Project\.venv\lib\site-packages\openrouteservice\client.py:211: User